# Digit Classifier

Based on MNIST dataset.
The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits
that is commonly used for training various image processing systems.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import shift
from matplotlib import pyplot as plt
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential 
from keras.layers.core import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D

c:\program files\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train = pd.read_csv("F:/Mnist datasets/train.csv")
test = pd.read_csv("F:/Mnist datasets/test.csv")

y_train = train["label"]

# Drop 'label' column from X_train
X_train = train.drop(labels = ["label"],axis = 1) 

#X_train = X_train.values.reshape(-1, 28, 28, 1) 
test = test.values.reshape(-1,28,28,1)

X_train = X_train.astype('float32')/255  #Divide by 255, so that we have pixel values between 0 and 1
test = test.astype('float32')/255

y_train = to_categorical(y_train, 10)  #The y_train vector has been converted such that it contains the 10 labels representing the digits.

In [5]:
def shift_digit(digit_array, dx, dy, new=0):
    return shift(digit_array.reshape(28,28), [dy, dx], cval=0).reshape(784)

Now, the original MNIST dataset contains 70000 images. These images are further divided into training and test sets. 
But this dataset is small. Hence we use data augmentation to slightly modify the images, so that we can increase the size of our
training and test sets.

Therefore, we have written a shift function which will shift the images by some pixels in horizontal and vertical directons.

More data means more accuracy.

In [7]:
X_train_augmented = [X_train]
y_train_augmented = [y_train]
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    shifted_images = np.apply_along_axis(shift_digit, axis=1, arr=X_train, dx=dx, dy=dy)
    X_train_augmented.append(shifted_images)
    y_train_augmented.append(y_train)

X_train_augmented = np.concatenate(X_train_augmented)
y_train_augmented = np.concatenate(y_train_augmented)
X_train_augmented.shape, y_train_augmented.shape

X_train_augmented = X_train_augmented.reshape(-1, 28, 28, 1)

In [8]:
#Three convolution layers

model = Sequential() 

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(256, activation='relu')) 
model.add(Dense(10, activation='softmax'))

model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(X_train_augmented, y_train_augmented, batch_size=60, epochs=7, verbose=1, validation_split=0.2) 

Train on 168000 samples, validate on 42000 samples
Epoch 1/7
168000/168000 [==============================] - 293s 2ms/step - loss: 0.0680 - acc: 0.9787 - val_loss: 0.0395 - val_acc: 0.9874
Epoch 2/7
168000/168000 [==============================] - 284s 2ms/step - loss: 0.0198 - acc: 0.9936 - val_loss: 0.0394 - val_acc: 0.9879
Epoch 3/7
168000/168000 [==============================] - 283s 2ms/step - loss: 0.0117 - acc: 0.9963 - val_loss: 0.0281 - val_acc: 0.9913
Epoch 4/7
168000/168000 [==============================] - 283s 2ms/step - loss: 0.0086 - acc: 0.9973 - val_loss: 0.0328 - val_acc: 0.9902
Epoch 5/7
168000/168000 [==============================] - 283s 2ms/step - loss: 0.0068 - acc: 0.9979 - val_loss: 0.0296 - val_acc: 0.9924
Epoch 6/7
168000/168000 [==============================] - 283s 2ms/step - loss: 0.0060 - acc: 0.9983 - val_loss: 0.0392 - val_acc: 0.9910
Epoch 7/7
168000/168000 [==============================] - 283s 2ms/step - loss: 0.0055 - acc: 0.9983 - val_loss: 0

In [10]:
import pandas as pd
# predict results
results = model.predict(test)

# select the index with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [11]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("F:/cnn_mnist_datagen.csv",index=False)